In [1]:
from __future__ import print_function
import ipywidgets as widgets 
import matplotlib.pyplot as plt
import numpy as np

# Non-Determinisitic Evaluation

### Mediator Parameters

$n$ = the number of times a jobs is executed for verification.

$PR$ = the penalty rate, used to determine the minimum deposit of both Job Creators and Resource Providers

$NDPR$ = the non-deterministic penalty rate, used to determine the Job Creators non-deterministic deposit

$available$ = The fee the Mediator requires to be available for a mediation. A mediator can be used if $available < JCmediatorPrice + RPmediatorPrice$

$mediationFee$ = The amount Mediator receives for mediating a given dispute. 

$ND\_Fee$ = The amount Mediator receives for mediating a given dispute and finding JC guilty of non-determinism. 

### Honest Mediator Model

In [2]:
# Mediator Parameters
wn = widgets.IntSlider(min=0,max=10,step=1,value=4, continuous_update=False)
wPR = widgets.FloatSlider(min=0,max=300,step=.5,value=1, description='PR', continuous_update=False) #Penalty Rate
wNDPR = widgets.FloatSlider(min=0,max=300,step=.5,value=1, description='NDPR',continuous_update=False)#Non-deterministic Penalty rate

def MNDU(JCmediatorPrice, RPmediatorPrice):
    available = JCmediatorPrice + RPmediatorPrice
    mediationFee = p*(p2+(1-q))*(p1**M_n)*(JCdeposit-price)
    ND_Fee = p*(p2+(1-q))*(1-(p1**M_n))*(NDdeposit-price) 
    return available + mediationFee + ND_Fee




### Solver Parameters

$matchFee$ = The amount a solver receives for finding a given match. 

### Honest Solver Model

In [3]:
#Solver
def SNDU(): #solver nondeterminisitc utility
    matchFee = RPMatchingFee + JCMatchingFee
    return matchFee

# JC False Job Model

This section analyzes when the JC is sending fake jobs that provide no value when they are successfully completed. The JC is attempting to gain utility though tricking the mediator into believing a RP did a job incorrectly and giving it compensation. The JC can trick the mediator by providing a job with a non-deterministic output. The output is in one of two answer classes, $a1$ or $a2$ with $P(a1)>>P(a2)$. If RP responds with $a1$ the JC will pay RP. If RP responds with $a2$ JC will request mediation, expecting the mediator to find the answer to be $a1$ and determine that the $RP$ must have cheated. 
We only need to considert two answer classes because when the Mediator is performing verification it assigns fault to the RP only if it finds there to be one answer which does not match the RP answer, if it finds two answers that differ from one another, regardless of if they match the RP answer, it faults the JC. This means that JC wants to maximize the chance that the Mediator will get the same answer reapeatedly which is not benefited by having additional answer classes.  

We need to make sure that the platform compensation model does not make this strategy feasible. 

**Assumptions**

* JC gets no benefit from the job
* The job has probability p1 of resulting in a1 and 1-p1 of resulting in a2
* Since JC knows both results it verifies every result at essentially 0 cost, pays RP for a1 and submits a2 for mediation. 

### Job Creator parameters

$price = instructionCount*JO.rate + bandwidthUsage*JO.rate$

$p$ = probability the JC verifies result.

$jcj$ = cost to JC to verify result.

$Benefit$ = the value JC gets when a job is performed correctly.

$matchPrice$ = the amount JC pays to Solver for finding a suitable RP. 

$mediatorPrice$ = The amount JC pays to mediator for being available. 

$p1$ = probability that job returns $a1$

$p2$ = $1-p1$ = probability that job returs $a2$

$deposit$ = the amount the Job Creator must post as a security deposit in order to get matched. 



In [4]:
# JC parameters
wPrice = widgets.IntSlider(min=0,max=10000,step=1,value=1, continuous_update=False)
JCgasPrice = 0
wp = widgets.FloatSlider(min=0,max=1,step=.1,value=1, description='P(verify)', continuous_update=False) # how often JC verifies
wjcj= widgets.FloatSlider(min=0,max=1,step=.1,value=0, description='jcj',continuous_update=False) # cost for JC to verify
wBenefit=widgets.IntSlider(min=0,max=10000,step=1,value=1, continuous_update=False)
matchPrice = 1
mediatorPrice = 1

def JCNDU(M_n,M_PR,M_NDPR, p1,price,p=1,jcj=0, q=1):
    p2 = 1-p1
    deposit = price * M_PR * M_n
    NDdeposit = price * M_NDPR
    
    noVerifyCost = -(1-p)*price
    
    verifyCost   = -p*jcj # chance to check job (p) and recompute (cj)
    A1payout     = -p*q*p1*price # RP correctly (q) returns answer 1 (p1) and pay up to max_price (-price).
    compensation =  p*q*p2*(p1**M_n)*(price + JCgasPrice) # receives gas price because it had to spend some to request mediation.
    NDfine       = -p*q*p2*(1-(p1**M_n))*NDdeposit #RP returns answer 2 correctly or responds incorrectly (p2+(1-q)) and send to mediator who finds out nonDeterminism (1-p1^n), pay fine and reward (f2+r)
    
    mediationCost= - p*(1-q)*JCgasPrice
    RPfraudC     =  p*(1-q)*(p1**M_n)*(price + JCgasPrice) 
    RPfraudL     = -p*(1-q)*(1-p1**M_n)*NDdeposit# compute incorrectly (1-q) get sent to mediation, finds nondeterminism (1-p1^n) get compensation.
    RPfraud1     =  RPfraudC + RPfraudL
    
    RPfraud2     = -p*(1-q)*price # JC doesn't want to risk getting caught, just pay RP. 
    
    JCnonDet = verifyCost + A1payout + compensation + NDfine + max(RPfraud1,RPfraud2) + mediationCost
    return JCnonDet

This model captures an honest Job Creator when $p1=1$, other values for $p1$ introduce non-deterministic cheating.

The Job Creators verification strategy is controlled by $p$, which is the probability of verifying a given result. 

The type of job, is captured by the $price$ and the $jcj$ terms. The $price$ capture the complexity of execution, $jcj$ captures the complexity of verification.

A mixed strategy Resource Provider is accounted for through the $q$ parameter. If $q=0$ then the $RP$ is intentionally sending junk results back. 

While running as a non-deterministic Job Creator it must decide whether the pay the $RP$ for a junk request, or risk getting caught in mediation. Currently the choice is made in the `max(RPfraud1,RPfraud2)` term. If the Job Creator is running deterministic jobs if it verifies and finds a bad result it will request mediation.

## Resource Providier

The Resource Provider only has one method to try to abuse the system. That is to return a result and hope that the Job Creator does not verify the job. If the Job Creator never verifies the Resource Provider will always cheat, and if it always verifies the Resource Provider will never cheat. However, the RP does not know how often the Job Creator will verify. 

### Resource Provider Model
The most RP can be paid for executing a job is $price = instructionCount*JO.rate + bandwidthUsage*JO.rate$. 

Jobs are only worth executing if the operating costs are covered by the Resource providers fees, which are at most $price$ for a given job. 

The Resource Provider does not only want to cover operating costs however, it additionally wants to make a profit. 

$profit = price-cost$

Since the price varies from job to job, the RP specifies a return on investment $roi$: 

$roi = \frac{price-cost}{price}$

For the model we need to estimate the cost of executing a job $cost$. We compute this with the given price and required roi. This is reasonable since the JC specifies the $price$ and $price$ is ralated to job complexity, and the execution cost will increase with complexity.

$cost = price*(1-roi)$ 

The platform will only match a job to an RP if $price>requestedPrice$
$requestedPrice$ covers the cost of execution. So in the analysis if $price<requestedPrice$ the point is infeasible. 
$requestedPrice$ is not used in the RP model, only computed. 

$requestedPrice = cost$

In a situation where the Job Creator is sending non-deterministic jobs, a Resource provider will lose 

$p \times p_2 \times p_1^n\times deposit$

If the RP assumes $p=1$(JC always verifies), and that JC uses optimal $p_1$ given $n$ and $NDPR$ it can determine the maximum percent $I$ of its deposit that it will lose to the mediator being tricked. By adding $ I \times deposit$ to its $requestedPrice$ it can mitigate the damage done by cheating Job Creators. $I$ is an insurance percent against cheating. This results in a new $requestedPrice$ :  

$requestedPrice = price*(1-roi) + deposit \times I $


When the RP is matched at its' $requestedPrice$ the $cost$ will effectively be reduced by $deposit \times I$ so the effective $cost$ used in the model is:

$cost = price*(1-roi) - deposit \times I $

In the `plot` function $requestedPrice$ is compared against $price$ to check if the point is valid. 


#### Honest RP Model

In [5]:
# RP parameters
RPgasPrice = 0
wq = widgets.FloatSlider(min=0,max=1,step=.1,value=1,description='P(run)', continuous_update=False) # probability that RP runs correctly
wroi = widgets.FloatSlider(min=0,max=1,step=.01,value=.5,description='roi', continuous_update=False) # what percent of reward is profit
winsurance = widgets.FloatSlider(min=0,max=1,step=.01,value=0 ,description='I',continuous_update=False) # % of deposit that can be recovered because execution is so cheap.
wrcj= widgets.FloatSlider(min=0,max=1,step=.1,value=0, description='rcj',continuous_update=False) # cost for RP to generate fake job response
matchPrice = 1
mediatorPrice = 1

def RPNDU(M_n,M_PR,M_NDPR, p1,price,p=1, q=1, roi=0, I=0):
    p2 = 1-p1
    deposit = price * M_PR * M_n
    
    execCost = - (price*(1-roi) - deposit*I)
 
    noVerifypayout = (1-p)*price #compute answer 1 (p1) correctly (q) receive reward (price)
    
    P1payout = p*p1*price #compute answer 1 (p1) correctly (q) receive reward (price)
    
    P2payout = p*p2*(1-p1**M_n)*price #compute answer 2 (p2) correctly get sent to mediation, who finds nondeterminism (1-p1^n)

    notP1cost = - p*(p2)*(p1**M_n)*deposit # compute answer 2 correctly or compute incorrectly (p2+1-q)  get sent to mediation, who doesn't find nondeterminsm (p1^n) and get fined (f)
    
#     failComp =   p*(1)*(1-p1**M_n)*damages# compute incorrectly (1-q) get sent to mediation, finds nondeterminism (1-p1^n) and bad RP? Pay RP for ND? Not sure its bad it could be totally random.
    
    RCnonDet = execCost + noVerifypayout + P1payout + P2payout + notP1cost
    return RCnonDet



#### Fraudulent model
The RP can decide to not execute the job correctly, and rather than paying the execution cost pay some lesser cost $Cj$. If RP does not compute the job it can receive $price$ if the JC does not verify, or chooses not to request mediation. It can receive $damages$ if the JC requests mediation and the Mediator finds non-determinism. 

In [6]:
# Resource Provider nondeterministic utility strategy D (decieving?), basically cheating
def RPNDUD(M_n,M_PR,M_NDPR,p1,price,p=1,rcj=0,q=1, roi=0, I=0):
    p2 = 1-p1
    deposit = price * M_PR * M_n
    
    execCost = - q*price*(1-roi) + deposit*I

    #Cj is cost of execution for fraudulant job, picking a random number or "close" value
    fraudCost = - (1-q)*rcj + deposit*I
    
    noVerifypayout = (1-p)*price #compute answer 1 (p1) correctly (q) receive reward (price)
    
    P1payout = p*q*p1*price #compute answer 1 (p1) correctly (q) receive reward (price)
    
    P2payout = p*q*p2*(1-p1**M_n)*price #compute answer 2 (p2) correctly (q) get sent to mediation, who finds nondeterminism (1-p1^n) and recieve reward (r) and nonDet bonus (f2-f)*k kickback=k

    P2cost = - p*q*p2*(p1**M_n)*deposit # compute answer 2 correctly (p2) get sent to mediation, who doesn't find nondeterminsm (p1^n) and get fined (f)
    
    fraud1C =   p*(1-q)*(1-p1**M_n)*price# compute incorrectly (1-q) get sent to mediation, finds nondeterminism (1-p1^n) get compensation.
    fraud1L = - p*(1-q)*(p1**M_n)*deposit #compute incorrectly (1-q) get sent to mediation, who doesn't find nondeterminsm (p1^n) and get fined (f)
    fraudPayout1 = fraud1C + fraud1L
    
    fraudPayout2 = p*(1-q)*price # JC doesn't want to risk getting caught, just pay RP. 
    
    
    RCnonDet = execCost + noVerifypayout + P1payout + P2payout + P2cost + fraudPayout1
    return RCnonDet

In [7]:
def plot(M_n,M_PR,M_NDPR,price,p=1,jcj=0, q=1,roi=1,Insurance=0,rcj=0):   
    plt.figure(2,figsize=(10,5))
    
    plt.subplot(1,1,1)
    plt.grid(visible=True)
    x,step = np.linspace(0,1,51,retstep=True)
    print("step size: %s" %step)
    yJND = list(map(lambda p1: JCNDU(M_n,M_PR,M_NDPR, p1,price,p=p,jcj=jcj, q=q),x))
    yRND = list(map(lambda p1: RPNDU(M_n,M_PR,M_NDPR, p1,price,p=p, q=q,roi=roi,I=Insurance),x))
    yRNDD = list(map(lambda p1: RPNDUD(M_n,M_PR,M_NDPR,p1,price,p=p,rcj=rcj,q=q,roi=roi,I=Insurance),x))
    plt.plot(x,yJND, label="JUND")
    plt.plot(x,yRND, label="RUND")
    plt.plot(x,yRNDD, label="RUNDD")
    plt.xlabel('P(a1)')
    plt.ylabel('Util') 
    
    plt.legend()
    plt.show
    
    print("n: %s" %M_n)
    print("Price: %s" %price)
    deposit = price * M_PR * M_n
    print("deposit: %s" %deposit)
    damages = price + RPgasPrice
    
    maxJND = max(yJND)
    p1Max = x[yJND.index(maxJND)]
    p2 = 1 - p1Max
    print("P(a1) for JCmax/RPmin util: %s" %p1Max)
    print("")
    
    print("Resource Provider")
    execCost = price*(1-roi)
    requstedPrice = execCost + deposit*Insurance   
    print("requstedPrice: %s" %requstedPrice)   
    print("price > requstedPrice, Matchable? : %s" %(price>requstedPrice))
    
    print("")
    P1payout = p*p1Max*price
    P2payout = p*p2*(1-p1Max**M_n)*damages
    TPayout = P1payout+P2payout
    RPcost = price*(1-roi) 
    NDI = deposit*Insurance
    NDloss = p*(p2)*(p1Max**M_n)*deposit
    Tcost = RPcost + NDloss - NDI
    I = p*(p2)*(p1Max**M_n)
    print("P1payout: %s" %P1payout)
    print("P2payout: %s" %P2payout)
    print("Total Payout: %s" %TPayout)
    print("")
    print("RPcost: -%s" %RPcost)
    print("RP NDloss: -%s" %NDloss)
    print("RP ND insurance: %s" %NDI)
    print("RP total expense: %s" %(-Tcost))
    print("min RP utility = %s" %(TPayout-Tcost))
    print("")
    print("min ROI for positive income: %s" %((price+NDloss-P1payout-P2payout)/price))
    print("Min insurance to cover ND losses: %s" %I)
    print("RP is losing: %s" %(min(yRND)<0))
    
    
    print("")
    print("")
    
    
    print("max ND JC utility: %s" %maxJND)    
   

In [8]:
# ui = widgets.HBox([wn,wPR,wNDPR,wPrice,wp,wcj,wq,wroi,winsurance])
ui1 = widgets.HBox([wn,wPR,wNDPR])
ui2 = widgets.HBox([wPrice,wp,wjcj])
ui3 = widgets.HBox([wrcj,wq,wroi,winsurance])
# interactive_plot = widgets.interactive(plot, M_n=wn,M_PR=wPR,M_NDPR=wNDPR,price=wPrice,p=wp,cj=wcj, q=wq,roi=wroi,Insurance=winsurance);
interactive_plot = widgets.interactive_output(plot,{'M_n':wn,'M_PR':wPR,'M_NDPR':wNDPR,'price':wPrice,'p':wp,'jcj':wjcj, 'q':wq,'roi':wroi,'Insurance':winsurance,'rcj':wrcj });
# output = interactive_plot.children[-1]
# output.layout.height = '900px'
display(interactive_plot,ui1,ui2,ui3)

Output()

# Benefit Analysis

This section analyzes when the JC is sending real jobs that provide it value when they are successfully completed. The JC is attempting to increase its utility by adding a non-deterministic piece to the jobs in an effort to have some jobs completed for free by accusing the RP of failing to execute them correctly. This could be done by changing the input so that some parts of the code are normally dorment and only activated when a certain data input is provided.

### Protocol Following model
If the JC follows the protocol its utility is the benefit, or value, it receives from the Job execution minus the amount it pays to the RP. This is assuming that the RP is playing by the rules of the game, otherwise the JC could have better utility since **price** will be refunded and **benefit** will still be recieved since the mediator will return the result. 

This model is further simplified by the assumption that a Job can be checked for free, and that the JC can check the result of every job. This is a reasonable simplification since there are jobs whose verification cost is much less than the cost of execution, and we are interested in identifying the worst case scenario we need to protect against. 

In [9]:
def protocolU(benefit, price):
    return benefit - price

### Non-Deterministic Model
The key element of this model is the amount of utility the JC can get for **free** from the system. 
The free utility is the probability that the RP got result 2 multiplied by the probability that the mediator got result 1 $n$ times multiplied by the job benefit, minus the probability that the RP got result 2 multiplied by the probability that the mediator did not get result 1 for any of $n$ executions multiplied by the deposit, whose value is determined by the price set by the JC multiplied by the non-deterministic penalty rate set by the Mediator. 

This model is the best the JC can hope for. The RP could fail to complete jobs, in which case the JC loses the benefit, and either pays $price$ or ND deposit.

In [10]:
def NDU(M_n,M_NDPR, p1,price,benefit):
    p2 = 1-p1
    NDdeposit = price * M_NDPR
    paid = p1*(benefit-price)
#     free = p2*((p1**M_n)*benefit - (1-p1**M_n)*NDdeposit)
    free = p2*((p1**M_n)*(benefit+price) - (1-p1**M_n)*NDdeposit)
    total = paid + free
    return total
    

### Evaluation

In [11]:
def plot2(M_n,M_NDPR,  price,benefit):
    x,step = np.linspace(0,1,51,retstep=True)
    yPU = list(map(lambda p1: protocolU(benefit,price),x))
    yNDU = list(map(lambda p1: NDU(M_n,M_NDPR, p1,price,benefit),x))
    
    
    plt.figure(3,figsize=(10,5))    
    plt.subplot(1,1,1)
    plt.grid(visible=True)
    plt.plot(x,yPU, label="PU")
    plt.plot(x,yNDU, label="NDU")
    plt.xlabel('P(a1)')
    plt.ylabel('Util') 
    
    plt.legend()
    plt.show
    
    print("Protocol util: %s" %yPU[0])
    print("max ND util: %s" %max(yNDU))
    print("ND improvement over Protocol: %s" %(max(yNDU)-yPU[0]))    
    print("%% improvement : %s %%" %(100*abs((max(yNDU)-yPU[0])/yPU[0])))
    print("optimal P(p1): %s" %x[yNDU.index(max(yNDU))])
    
    

In [12]:
interactive_plot2 = widgets.interactive(plot2, M_n=wn,M_NDPR=wNDPR, price=wPrice,benefit=wBenefit);
output = interactive_plot2.children[-1]
output.layout.height = '600px'
interactive_plot2

#M_n = 4
#NDPR = 24? if price is included in free job.

interactive(children=(IntSlider(value=4, continuous_update=False, description='M_n', max=10), FloatSlider(valu…

### Bounding the problem
$free = (1-P)P^nb - (1-P)(1-P^n)Rc$

**Derivative**

$\frac{d}{dP}(free) = \frac{d}{dP} (1-P)P^nb - (1-P)(1-P^n)Rc$

$0 = \frac{d}{dP} (P^nb - P^{n+1} - Rc + P^nRc + PRc - P^{n+1}Rc$)

$0 = nP^{n-1}b - (n+1)P^{n}b - 0 + nP^{n-1}Rc + Rc - (n+1)P^{n}Rc$

**Collect Terms**

$ nP^{n-1}b + nP^{n-1}Rc + Rc =  (n+1)P^{n}b + (n+1)P^{n}Rc$

$ nP^{n-1}(b+Rc) + Rc =  (n+1)P^{n}(b+Rc)$

$ nP^{n-1} + \frac{Rc}{b+Rc} =  (n+1)P^{n}$


$\frac{Rc}{b+Rc} + nP^{n-1} - (n+1)P^n = 0 $

**Let penalty rate R=0**

benefit and cost become irrelevant and we are left with:

$nP^{n-1}  = (n+1)P^n $

**Simplify**

$\frac{n}{n+1} = P$

So the maximum the JC can make with no penalty rate will be when $P = \frac{n}{n+1}$

$P$ is the liklihood of getting answer 1 and $n$ is the number of times the mediator verifies, so we can force the JC to cheat less and limit its additional utility. 

For example if $n=4$ then $P=.8$, so for optimal gains JC will make it so that a1 appears 80% of the time, anything else would give it less reward, so if we assume the worst case and handle it, anything less severe will be handled as well. 

**If R is not 0**

any change to $b$ or $c$ will never make lower probability of a1 more optimal. 

**For $b<c$ and R=1**

The normal protocol loses, but the ND protocol loses somewhat less until $b=.9c$ approximately, as which point ND can win slightly. 

It can win .694% utility units when $b=90$ and $c=100$.

**For $b=c$ and R=1**

Optimal is when probability of a1 is 90%. 
Normal protocal loses, and ND protocol wins 9.683% of benfit(or price since they are equal) as utility. 

**For $b>c$ and R=1**

As $b$ grows the advantage over the protocol decreases compared to $b=c$

When $b=4.87c$ the added benefit is less than 1%. 

**Things get worse for JC if R increases.**
The worst case for the system is $b=c$ as the JC can get the most advantage. This can be mitigated by assuming JC is cheating and taking a 9.683% tax on the price it includes, or increasing R to, apparently, 24, after which point ND can't win and is only better than protocol if $b=.762c$ or less (meaning its losing badly since it needed 90% to win in the first place).




Taha's email : 

* Docker Layers
    * RP will have a function call resourceProviderAddDockerLayer which he uses to specify his supported 1st-Layers.
    * JC will specify the first layer when he posts a job offer.
    * A job offer and a resource offer are matchable if the job offer's first docker layer is in the RP's first layers list.
* Penalty
        The security deposit of the cheater will stay in the platform instead of being rewarded to the mediation winner.
        RP can accept a mediation verdict or can ask for extra rounds of mediation. In each round, he will pay the mediation value. (Is this money as the same as last run's cost?)
        Winner of the mediation verdict will receive Mediator's last instructionCount and bandwidthUsage times his prices for these two units as a reward. If the winner is the RP, he will receive the cost of all rounds of mediation.
    Solver
        RP and JC specify a solver cost when they post an offer. It will be compensated for the first Solver which match the offer. (This will decrease the efficiency of the matches unless we have an offering period and matching period like Transax. Then, the contract can choose the best matches and only reward those solvers who posted the best solution.)
    Mediator
        Mediators specify a price for availability. They both have to pay for this value because they trusted it.
        Are we going to have Mediators accountable?